In [1]:
!pip install langchain==0.1.0
!pip install openai
!pip install streamlit
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken

In [3]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [17]:
import os
os.environ["OPENAI_API_KEY"]= "***************************************"

In [48]:
pdfreader = PdfReader(r"C:\Users\shash\regulations.pdf")

In [49]:
from typing_extensions import Concatenate
raw_text = " "
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [50]:
raw_text

'  \n \nRegulations Governing the Degree of  \nBachelor  of Engineering  (B.E.)  \n \nEffective  from the Academic  Year  2021 -22 \n \n \n \n \n \n \n \n \n \nDr. Ambedkar  Institute  of Technology,  Bangalore  \n[ Estd. 1980 (An Autonomous Institution, affiliated to VTU, Belagavi,)  \nApproved  by AICT E, New  Delhi,  Grant  in Aid Institution  (Govt.  of \nKarnataka,) Accredited by NBA & NAAC with ‘A’ grade]  \nBDA  Outer  Ring  Road,  Mallathahalli,  Bengaluru  - 560 056 \nOuter  Ring  Road,  Mallathahalli,  Bengaluru - 560 056 \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n1 \n2   \nDr. Ambedkar  Insti tute of Technology  \nCONTENTS  \nSection  Particulars  Page  \n1 Short  Title  and Commencement  3 \n2 Definitions  3 \n3 Preamble  3-4 \n4 Academic  Programmes  4 \n 4.1 General  4-5 \n 4.2 Nomenclatures  of Programmes  5-6 \n 4.3. Programmes  Offered  at Dr. AIT 6-7 \n 4.4 Programme  Duration  7-8 \n 4.5 Admission  of Students  8-9 \n 4.6 Semester  S

In [51]:
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size = 800,
    chunk_overlap = 200,
    length_function = len)
texts = text_splitter.split_text(raw_text)

In [52]:
texts[0]

'Regulations Governing the Degree of  \nBachelor  of Engineering  (B.E.)  \n \nEffective  from the Academic  Year  2021 -22 \n \n \n \n \n \n \n \n \n \nDr. Ambedkar  Institute  of Technology,  Bangalore  \n[ Estd. 1980 (An Autonomous Institution, affiliated to VTU, Belagavi,)  \nApproved  by AICT E, New  Delhi,  Grant  in Aid Institution  (Govt.  of \nKarnataka,) Accredited by NBA & NAAC with ‘A’ grade]  \nBDA  Outer  Ring  Road,  Mallathahalli,  Bengaluru  - 560 056 \nOuter  Ring  Road,  Mallathahalli,  Bengaluru - 560 056 \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n1 \n2   \nDr. Ambedkar  Insti tute of Technology  \nCONTENTS  \nSection  Particulars  Page  \n1 Short  Title  and Commencement  3 \n2 Definitions  3 \n3 Preamble  3-4 \n4 Academic  Programmes  4 \n 4.1 General  4-5'

In [53]:
texts[1]

'Dr. Ambedkar  Insti tute of Technology  \nCONTENTS  \nSection  Particulars  Page  \n1 Short  Title  and Commencement  3 \n2 Definitions  3 \n3 Preamble  3-4 \n4 Academic  Programmes  4 \n 4.1 General  4-5 \n 4.2 Nomenclatures  of Programmes  5-6 \n 4.3. Programmes  Offered  at Dr. AIT 6-7 \n 4.4 Programme  Duration  7-8 \n 4.5 Admission  of Students  8-9 \n 4.6 Semester  Scheme  9-10 \n5 Credit  System  10 \n 5.1 General  10-12 \n 5.2 Credit  Structure  12 \n 5.3 Course  Load  12-13 \n 5.4 Prescribed Number of Credits for the Progra mme  13 \n 5.5 Contact Hour s 13-14 \n 5.6 Course Registration Details  14 \n 5.7 Supplementary semester  14-15 \n6 Curriculum  Framework  15 \n 6.1 General  Issues  15-16 \n 6.2 B.E. Degree  Programme  16-19 \n7 Assessment  19 \n 7.1 Achievement Testing  19'

In [54]:
embeddings = OpenAIEmbeddings()

In [55]:
document_search = FAISS.from_texts(texts,embeddings)

In [56]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [57]:
chain = load_qa_chain(OpenAI(), chain_type = "stuff")

In [58]:
query  = "Explain the Gracing Policy"
docs = document_search.similarity_search(query)
chain.run(input_documents = docs, question = query)

' The gracing policy allows for students to receive a maximum of 5 grace marks in a subject if they have failed in only one subject or course. This can be applied to both theoretical and practical exams, and can also be used to declare a student eligible for a higher class or degree. The grace marks will be added to the total aggregate marks of the student, and the student will be considered to have passed in a higher class. However, a student is only entitled to grace marks under one of the regulations (GMR01, GMR02, or GMR03), and the grace marks are only shown in the ledger and not on the statement of marks. The gracing policy also takes into account attendance and the participation of students in extracurricular activities. '

In [61]:
query  = "Give the definitions"
docs = document_search.similarity_search(query)
chain.run(input_documents = docs, question = query)

' \na) Academic Autonomy - freedom granted by the University to a College in all aspects of conducting its academic programmes for promoting excellence.\nb) Autonomous College - a College notified as an autonomous college by the University as per the VTU Statutes on Autonomous Colleges (Amended) 2011 and further amended from time to time as per UGC Regulations/Guidelines.\nc) Commission - University Grants Commission.\nd) Council - All India Council for Technical Education.'

In [63]:
query  = "What are the academic programs offered ?"
docs = document_search.similarity_search(query)
chain.run(input_documents = docs, question = query)

' The academic programs offered include B.E at Undergraduate (UG), M.Tech., M.B.A, M.C.A, M.Sc. (Engineering by research), and Ph.D.'

In [65]:
query  = "What are the programmes offered at Dr.AIT?"
docs = document_search.similarity_search(query)
chain.run(input_documents = docs, question = query)

' The programmes offered at Dr.AIT are Bachelor of Engineering (B.E) at the Undergraduate level, Master of Technology (M.Tech.), Master of Business Administration (M.B.A.), Master of Computer Applications (M.C.A.) at the Postgraduate level, and M.Sc. (Engineering) by Research and Doctor of Philosophy (Ph.D.) at the Research level. The specific branch and subject of specialization may be indicated in brackets after the abbreviation.'

In [66]:
query  = "What is Dr.AIT?"
docs = document_search.similarity_search(query)
chain.run(input_documents = docs, question = query)

' Dr. AIT stands for Dr. Ambedkar Institute of Technology. It is an institution that offers academic programs and follows a credit system for assessment. It also allows for transfer of students from other autonomous institutions and temporary withdrawal from the program under certain circumstances.'